In [1]:
import gc
import numpy as np
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import utils, segmentation, AL, presentation
from tqdm.auto import tqdm

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:1


In [3]:
ds_full = segmentation.SegmentationDataset('./cut_stenoses_data/train/', 2)
ds_test = segmentation.SegmentationDataset('./cut_stenoses_data/test/', 2)

batch_size = 8
dl_full = torch.utils.data.DataLoader(ds_full, batch_size=batch_size, shuffle=False, drop_last=False)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [4]:
# w = 0.
# for imgs, masks in dl_full:
#     w += masks.sum() / (masks.numel() * len(dl_full))
# loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1 / (1-w), 1/ w])).to(device)
loss_fn = nn.CrossEntropyLoss()

In [5]:
model = presentation.create_model().to(device)
optimizer = presentation.create_optimizer(model)

In [6]:
N = len(ds_full)
N_step = int(N*0.1)
N, N_step

(598, 59)

In [7]:
idxs = list(range(N))
np.random.seed(0)
np.random.shuffle(idxs)

In [8]:
for n_iter in range(10):
    print(f'>>>>>>>>>>>> Iter {n_iter}')
    del model
    del optimizer
    gc.collect()
    labeled_idxs = idxs[:N_step*(n_iter+1)]
    ds_train = AL.PartialDs(ds_full, labeled_idxs)
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)

    model = presentation.create_model().to(device)
    optimizer = presentation.create_optimizer(model)

    num_epochs = 30
    for epoch in tqdm(range(num_epochs), desc='Epoch'):
        trn_loss = utils.train(dl_train, loss_fn, model, optimizer)
        val_loss, val_IoUs = segmentation.evaluate(dl_test, loss_fn, model)
        print(epoch, f'{trn_loss:.4f}', f'{val_loss:.4f}', val_IoUs)
    print()

>>>>>>>>>>>> Iter 0


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.1318 0.0585 tensor([0.])
1 0.0187 0.0302 tensor([0.])
2 0.0136 0.0269 tensor([0.])
3 0.0112 0.0245 tensor([0.])
4 0.0108 0.0243 tensor([0.])
5 0.0092 0.0267 tensor([0.])
6 0.0081 0.0274 tensor([0.])
7 0.0077 0.0294 tensor([0.])
8 0.0078 0.0288 tensor([0.])
9 0.0071 0.0298 tensor([0.])
10 0.0069 0.0289 tensor([0.])
11 0.0071 0.0288 tensor([0.])
12 0.0067 0.0314 tensor([0.])
13 0.0064 0.0311 tensor([0.])
14 0.0062 0.0310 tensor([0.])
15 0.0062 0.0318 tensor([0.])
16 0.0060 0.0322 tensor([0.])
17 0.0060 0.0316 tensor([0.])
18 0.0059 0.0338 tensor([0.])
19 0.0058 0.0329 tensor([0.])
20 0.0058 0.0346 tensor([0.])
21 0.0057 0.0335 tensor([0.])
22 0.0056 0.0342 tensor([0.])
23 0.0055 0.0339 tensor([0.])
24 0.0055 0.0350 tensor([0.])
25 0.0054 0.0350 tensor([0.])
26 0.0053 0.0338 tensor([0.])
27 0.0054 0.0347 tensor([0.])
28 0.0052 0.0354 tensor([0.])
29 0.0052 0.0351 tensor([0.])

>>>>>>>>>>>> Iter 1


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.1129 0.0288 tensor([0.])
1 0.0160 0.0220 tensor([0.])
2 0.0120 0.0235 tensor([0.])
3 0.0099 0.0217 tensor([0.])
4 0.0087 0.0202 tensor([0.])
5 0.0077 0.0231 tensor([0.])
6 0.0073 0.0196 tensor([0.])
7 0.0072 0.0205 tensor([0.])
8 0.0068 0.0220 tensor([0.])
9 0.0065 0.0193 tensor([0.])
10 0.0063 0.0236 tensor([0.])
11 0.0061 0.0228 tensor([0.])
12 0.0059 0.0217 tensor([0.])
13 0.0057 0.0226 tensor([0.])
14 0.0057 0.0232 tensor([0.])
15 0.0056 0.0232 tensor([0.])
16 0.0054 0.0232 tensor([0.])
17 0.0053 0.0234 tensor([0.])
18 0.0052 0.0215 tensor([0.])
19 0.0051 0.0210 tensor([0.])
20 0.0049 0.0213 tensor([0.])
21 0.0048 0.0217 tensor([0.0004])
22 0.0047 0.0214 tensor([0.2400])
23 0.0046 0.0202 tensor([0.2828])
24 0.0046 0.0207 tensor([0.2961])
25 0.0045 0.0217 tensor([0.3009])
26 0.0045 0.0208 tensor([0.3123])
27 0.0043 0.0205 tensor([0.3283])
28 0.0043 0.0187 tensor([0.3303])
29 0.0042 0.0211 tensor([0.3179])

>>>>>>>>>>>> Iter 2


...

0 0.0247 0.0215 tensor([0.])
1 0.0077 0.0222 tensor([0.])
2 0.0063 0.0230 tensor([0.])
3 0.0056 0.0203 tensor([0.])
4 0.0051 0.0215 tensor([0.2519])
5 0.0047 0.0226 tensor([0.2996])
6 0.0044 0.0214 tensor([0.3359])
7 0.0041 0.0222 tensor([0.3440])
8 0.0039 0.0194 tensor([0.3776])
9 0.0037 0.0180 tensor([0.4041])
10 0.0035 0.0230 tensor([0.3626])
11 0.0033 0.0219 tensor([0.3789])
12 0.0031 0.0209 tensor([0.3923])
13 0.0030 0.0221 tensor([0.3893])
14 0.0029 0.0227 tensor([0.3913])
15 0.0028 0.0213 tensor([0.4050])
16 0.0027 0.0211 tensor([0.4120])
17 0.0026 0.0221 tensor([0.4033])
18 0.0025 0.0211 tensor([0.4254])
19 0.0024 0.0204 tensor([0.4276])
20 0.0023 0.0216 tensor([0.4249])
21 0.0022 0.0225 tensor([0.4176])
22 0.0022 0.0217 tensor([0.4214])
23 0.0021 0.0223 tensor([0.4298])
24 0.0020 0.0204 tensor([0.4360])
25 0.0020 0.0217 tensor([0.4342])
26 0.0019 0.0230 tensor([0.4258])
27 0.0019 0.0213 tensor([0.4415])
28 0.0018 0.0218 tensor([0.4396])
29 0.0018 0.0214 tensor([0.4396])

>>>>>

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.0264 0.0223 tensor([0.])
1 0.0076 0.0234 tensor([0.])
2 0.0064 0.0224 tensor([0.])
3 0.0056 0.0216 tensor([5.3333e-05])
4 0.0052 0.0203 tensor([0.1929])
5 0.0047 0.0168 tensor([0.3121])
6 0.0044 0.0192 tensor([0.2932])
7 0.0041 0.0203 tensor([0.3240])
8 0.0039 0.0196 tensor([0.3307])
9 0.0036 0.0184 tensor([0.3679])
10 0.0034 0.0197 tensor([0.3531])
11 0.0032 0.0207 tensor([0.3508])
12 0.0030 0.0202 tensor([0.3717])
13 0.0029 0.0226 tensor([0.3495])
14 0.0028 0.0153 tensor([0.4575])
15 0.0027 0.0208 tensor([0.3771])
16 0.0025 0.0208 tensor([0.3828])
17 0.0024 0.0193 tensor([0.4089])
18 0.0024 0.0206 tensor([0.3939])
19 0.0022 0.0172 tensor([0.4440])
20 0.0022 0.0197 tensor([0.4142])
21 0.0022 0.0195 tensor([0.4154])
22 0.0020 0.0170 tensor([0.4472])
23 0.0020 0.0192 tensor([0.4323])
24 0.0019 0.0216 tensor([0.4056])
25 0.0019 0.0186 tensor([0.4429])
26 0.0017 0.0196 tensor([0.4379])
27 0.0017 0.0190 tensor([0.4456])
28 0.0017 0.0170 tensor([0.4695])
29 0.0016 0.0193 tensor([0.4422]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0 0.0307 0.0169 tensor([0.])
1 0.0071 0.0156 tensor([0.])
2 0.0062 0.0172 tensor([0.])
3 0.0054 0.0173 tensor([0.])
4 0.0049 0.0150 tensor([0.3602])
5 0.0045 0.0177 tensor([0.3792])
6 0.0042 0.0166 tensor([0.4174])
7 0.0039 0.0151 tensor([0.4492])
8 0.0036 0.0148 tensor([0.4581])
9 0.0034 0.0177 tensor([0.4312])
10 0.0032 0.0165 tensor([0.4523])
11 0.0030 0.0167 tensor([0.4587])
12 0.0029 0.0170 tensor([0.4554])
13 0.0027 0.0167 tensor([0.4604])
14 0.0026 0.0170 tensor([0.4635])
15 0.0025 0.0163 tensor([0.4699])
16 0.0023 0.0152 tensor([0.4889])
17 0.0023 0.0157 tensor([0.4877])
18 0.0022 0.0166 tensor([0.4803])
19 0.0021 0.0177 tensor([0.4581])
20 0.0021 0.0159 tensor([0.4935])
21 0.0019 0.0154 tensor([0.5016])
22 0.0019 0.0151 tensor([0.5149])
23 0.0018 0.0149 tensor([0.5210])
24 0.0018 0.0148 tensor([0.5159])
25 0.0017 0.0154 tensor([0.5127])
26 0.0016 0.0152 tensor([0.5166])
27 0.0016 0.0159 tensor([0.5072])
28 0.0016 0.0159 tensor([0.5129])
29 0.0015 0.0156 tensor([0.5163])

